## Análise de dados das inscrições para o concurso público da SEMED para a Prefeitura de Maricá

### Este projeto visa coletar, tratar e analisar os dados da Lista Geral Preliminar das inscrições confirmadas para o concurso público da SEMED para a Prefeitura de Maricá
### Url do arquivo: https://portal.coseac.uff.br/wp-content/uploads/sites/596/2024/04/Concurso-PMM-SEMED20241-ListaGeralPreliminardasinscricoesconfirmadas.pdf
### Data de acesso: 02 de maio de 2024

## Análise inicial do arquivo pdf
### Número de páginas: 1994
### Característiras: Todas as páginas apresentam-se padronizadas com a seguite estrutura:
### - Cabeçalho;
### - Conteúdo;
### - Rodapé com legendas.

## Bibliotecas Python utilizadas:
### - pdfplumber: para a leitura do arquivo
### - pandas: para a limpeza e tratamento dos dados

### Código para auxílio na análise exploratória 

In [1]:
import pandas as pd
import pdfplumber

### O código abaixo erifica se os cabeçalhos de todas as páginas são iguais

In [ ]:
path_to_file = "Concurso-PMM-SEMED20241-ListaGeralPreliminardasinscricoesconfirmadas.pdf"

with pdfplumber.open(path_to_file) as pdf:
    page_0_header = [pdf.pages[0].extract_text().split("\n")[i] for i in range(8) if i != 2]
    
    for i in range(0, 1994):
        page = pdf.pages[i]
        text = page.extract_text()
        page_x_header = [text.split("\n")[i] for i in range(8) if i != 2]
        
        if page_0_header != page_x_header:
            page_number = text.split("\n")[2]
            advise = f'There is diferent header on the page {page_number}'
            print(advise)
            break
    
    print("All headers are the same!")

### O código cria e preenche abaixo uma lista que será usada na geração do dataframe inicial.

In [2]:
path_to_file = "Concurso-PMM-SEMED20241-ListaGeralPreliminardasinscricoesconfirmadas.pdf"
all_lines_to_dataframe = [["Cargo", "TipoVaga"]]

In [10]:
with pdfplumber.open(path_to_file) as pdf:
       
    for i in range(0, 1994): #definir o intervalo das páginas
        page = pdf.pages[i]
        text = page.extract_text()
        
        if "Inscritos: 69770" in text: #Verifica se é a última página
            for i in range(8, 23):
                unformated_line = text.split('\n')[i].split(' ')
                docente_position = 0
                for j in range(len(unformated_line)):
                    if "Docente" in unformated_line[j]:
                        docente_position = j
            
                all_lines_to_dataframe.append([" ".join(unformated_line[docente_position:-1]), unformated_line[-1]])
        else:
            counter = 0
            limit_line_number = 0
            for line in text.split('\n'):
                counter = counter + 1
                if "Tipo de Vaga" in line:
                    limit_line_number = counter
           
            for i in range(8, limit_line_number-1):
                unformated_line = text.split('\n')[i].split(' ')
                docente_position = 0
                for j in range(len(unformated_line)):
                    if "Docente" in unformated_line[j]:
                        docente_position = j
            
                all_lines_to_dataframe.append([" ".join(unformated_line[docente_position:-1]), unformated_line[-1]])

In [15]:
df = pd.DataFrame(all_lines_to_dataframe)
df.columns = df.iloc[0]
df = df[1:]
df

,Cargo,TipoVaga
1,Docente I - Geografia,AC
2,Docente I - Ciências Físicas e Biológicas,AC
3,Docente I - Geografia,AC
4,Docente I - História,AC
5,Docente I - Educação Física,AC
...,...,...
69766,Docente I - História,AC
69767,Docente II,AC
69768,Docente II,AC
69769,Docente II,AC


### Criação de uma arquivo .cvs com os dados sobre Cargo e Tipo de Vaga

In [16]:
df.to_csv('Marica_Cargo_TipoVaga.csv')